<a href="https://colab.research.google.com/github/ahoiching/FAM/blob/main/FAM_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%shell
pip install pandas --upgrade
pip install scipy --upgrade
pip install tqdm --upgrade
pip install --upgrade cvxpy
pip install cobra

wget https://packages.gurobi.com/9.5/gurobi9.5.0_linux64.tar.gz -O /opt/gurobi9.5.0_linux64.tar.gz
tar xfz /opt/gurobi9.5.0_linux64.tar.gz -C /opt/
pip install gurobipy==9.5.0

--2022-05-20 05:15:16--  https://packages.gurobi.com/9.5/gurobi9.5.0_linux64.tar.gz
Resolving packages.gurobi.com (packages.gurobi.com)... 13.35.118.2, 13.35.118.102, 13.35.118.93, ...
Connecting to packages.gurobi.com (packages.gurobi.com)|13.35.118.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73784264 (70M) [application/x-gzip]
Saving to: ‘/opt/gurobi9.5.0_linux64.tar.gz’

/opt/gurobi9.5.0_li 100%[===================>]  70.37M  85.7MB/s    in 0.8s    

2022-05-20 05:15:17 (85.7 MB/s) - ‘/opt/gurobi9.5.0_linux64.tar.gz’ saved [73784264/73784264]



In [6]:
!/opt/gurobi950/linux64/bin/grbgetkey YOUR_GUROBI_LICENSE_KEY

info  : grbgetkey version 9.5.0, build v9.5.0rc5
info  : Contacting Gurobi license server...
info  : License file for license ID 803110 was successfully retrieved
info  : License expires at the end of the day on 2022-07-01
info  : Saving license file...

In which directory would you like to store the Gurobi license file?
[hit Enter to store it in /opt/gurobi]: 

info  : License 803110 written to file /opt/gurobi/gurobi.lic


In [7]:
!git clone https://github.com/ahoiching/FAM.git

Cloning into 'FAM'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 7), reused 8 (delta 3), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [8]:
%cd ./FAM/
import FAM
import pandas as pd
import numpy as np

/content/FAM


In [11]:
(flux_modes, flux_modes_curate, flux_decomposition, functional_decomposition,
  gene_expressions_decomposition)=FAM.Get_yield_flux_functional_gene_decomposition_from_file('./model_example.json')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-07-01


100%|██████████| 1877/1877 [00:00<00:00, 486961.63it/s]



.
.
Metabolites loaded...


100%|██████████| 2791/2791 [00:00<00:00, 14895.30it/s]



.
.
Reactions loaded...


...


.
.
Getting gene reactions matrix from FBA model...

.
.
Genes loaded...


100%|██████████| 2791/2791 [00:00<00:00, 373931.59it/s]



.
.
Reactions loaded...
Reactions that are constraint to carry fixed non-zero fluxes:
Lower bound		Upper bound 		Reaction ID 
5.3024922118  		 5.3024922118 		 EX_ac_e
72.1076287777  		 72.1076287777 		 ATPM
0.130505201  		 0.130505201 		 Sink_RNA|atp_c
0.1214848392  		 0.1214848392 		 Sink_RNA|ctp_c
0.1643765566  		 0.1643765566 		 Sink_RNA|gtp_c
0.1067840295  		 0.1067840295 		 Sink_RNA|utp_c
0.5564993963  		 0.5564993963 		 Sink_Protein|ala__L_c
0.294656916  		 0.294656916 		 Sink_Protein|arg__L_c
0.2253865467  		 0.2253865467 		 Sink_Protein|asn__L_c
0.3195778331  		 0.3195778331 		 Sink_Protein|asp__L_c
0.0440936425  		 0.0440936425 		 Sink_Protein|cys__L_c
0.2132579501  		 0.2132579501 		 Sink_Protein|gln__L_c
0.3612168677  		 0.3612168677 		 Sink_Protein|glu__L_c
0.4320056384  		 0.4320056384 		 Sink_Protein|gly_c
0.102966245  		 0.102966245 		 Sink_Protein|his__L_c
0.31296223  		 0.31296223 		 Sink_Protein|ile__L_c
0.4690364829  		 0.4690364829 		 Sink_Protein|leu__L_c
0.345509

100%|██████████| 81/81 [00:36<00:00,  2.22it/s]



..
The condition is identified as aerobic E.coli..
..
..



In [34]:
protein_mass_fractions=pd.read_csv('./example_protein_mass_fractions.csv', 
                                   index_col=0)
shared_index=np.intersect1d(protein_mass_fractions.index, 
                            gene_expressions_decomposition.index)
non_FAM_index=np.setdiff1d(protein_mass_fractions.index, 
                          gene_expressions_decomposition.index)

In [29]:
FAM_decomposition=gene_expressions_decomposition.loc[shared_index].multiply(
    protein_mass_fractions.loc[shared_index,'mass fractions'], 
                                                          axis=0)

In [33]:
FAM_decomposition=FAM_decomposition.rename(columns={"EX_ac_e": "Energy|Fermentation", 
                                  "ATPM": "Energy|Respiration",
                                  "__RESIDUALS__": "No flux"})

FAM_decomposition

,Energy|Fermentation,Energy|Respiration,Sink_RNA|atp_c,Sink_RNA|ctp_c,Sink_RNA|gtp_c,Sink_RNA|utp_c,Sink_Protein|ala__L_c,Sink_Protein|arg__L_c,Sink_Protein|asn__L_c,Sink_Protein|asp__L_c,...,Sink_Others|ribflv_c,Sink_Others|sheme_c,Sink_Others|so4_c,Sink_Others|thf_c,Sink_Others|thmpp_c,Sink_Others|udcpdp_c,Sink_Others|zn2_c,Sink_Others|gthrd_c,No flux,__MIX_FUNCTIONS__
b0002,8.654303e-16,1.985343e-16,4.807531e-16,4.932551e-16,9.168445e-16,9.853240e-16,1.082729e-15,1.060910e-15,6.046455e-16,6.405738e-16,...,4.269017e-18,4.273714e-18,4.270801e-18,4.269867e-18,4.269051e-18,4.269724e-18,0.000000e+00,0.000000,3.553614e-16,5.094291e-14
b0003,1.261503e-14,1.149472e-18,0.000000e+00,0.000000e+00,0.000000e+00,9.189471e-15,2.007956e-14,3.405276e-14,5.408312e-15,0.000000e+00,...,3.669847e-14,1.398816e-13,3.506733e-14,6.654507e-08,1.345322e-14,1.093845e-14,3.273088e-14,0.000000,1.047391e-16,2.234383e-12
b0004,3.676960e-18,3.583341e-18,0.000000e+00,0.000000e+00,0.000000e+00,5.043051e-17,1.294895e-16,1.140261e-16,0.000000e+00,0.000000e+00,...,8.154400e-18,8.154732e-18,8.154526e-18,8.154460e-18,8.154402e-18,8.154450e-18,0.000000e+00,0.000000,0.000000e+00,2.622046e-15
b0007,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
b0008,2.553102e-13,1.640845e-04,6.422618e-05,5.231358e-05,9.100739e-05,4.598314e-05,0.000000e+00,2.853025e-13,2.773015e-05,3.931886e-05,...,5.180115e-08,9.496922e-08,2.584988e-13,6.906838e-08,6.043567e-08,2.342291e-08,0.000000e+00,0.000009,0.000000e+00,5.630620e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4481,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.170000e-05,0.000000e+00
b4485,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
b4513,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
b4515,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00


In [52]:
GO_genes=pd.concat([protein_mass_fractions.loc[non_FAM_index,'mass fractions'],
                    FAM_decomposition['No flux']], axis=0).sort_index()
GO_genes=GO_genes[GO_genes>1e-6]

GO_genes

b0006    0.000106
b0009    0.000058
b0014    0.005736
b0015    0.000338
b0019    0.000053
           ...   
b4536    0.000021
b4537    0.000065
b4554    0.000027
b4555    0.000004
b4558    0.000085
Length: 1602, dtype: float64